# Ładowanie modeli
Ładowanie dwóch modeli, które nauczyliśmy wraz z ich najlepszymi wagami.

In [1]:
from keras.models import *
from custom_layers import BatchNormalization, Softmax4D

segnet_path = "modeleAF/model-e6dc8464-7322-4237-b669-95e4ad5819cb/"
#https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/
unet_path = "model-a679583b-da1c-437e-aaa4-58ef1f9e4f58/"

#load models
with open(segnet_path + 'architecture.json', 'r') as json:
    segnet_load = json.read()
segnet = model_from_json(segnet_load, {"Softmax4D": Softmax4D, 
                                       "BatchNormalization": BatchNormalization})

with open(unet_path + 'architecture.json', 'r') as json:
    unet_load = json.read()
    
unet = model_from_json(unet_load, {"Softmax4D": Softmax4D})

In [2]:
#load weights
segnet.load_weights(segnet_path + "weights.87-0.98.hdf5")
unet.load_weights(unet_path + "weights.15-0.78.hdf5")

In [3]:
segnet.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, None, None, 3)     0         
_________________________________________________________________
conv2d (Conv2D)              (None, None, None, 64)    1792      
_________________________________________________________________
batch_normalization (BatchNo (None, None, None, 64)    256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, None, None, 64)    0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, None, None, 64)    0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, None, None, 128)   73856 

In [4]:
unet.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 512, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 512, 512, 64) 36928       conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 256, 256, 64) 0           conv2d_1[0][0]                   
______________________________________________________________________________________________

*Segnet* osiągnał powyżej 0.97 ważonej celności, a *U-Net* 0.78.

In [5]:
segnet.trainable = False
len(segnet.trainable_weights)

0

In [6]:
unet.trainable = False
len(unet.trainable_weights)

0

In [7]:
def change_layers_names(model, name):
    for layer in model.layers:
        layer._name = name + "_" + layer.name
        
change_layers_names(segnet, "segnet")
change_layers_names(unet, "unet")

In [8]:
from keras.layers import Concatenate, Conv2D

ensemble_input = [segnet.input, unet.input]
ensemble_output = [segnet.input, unet.input]

merge = Concatenate()(ensemble_output)
conv = Conv2D(2, (1, 1))(merge)
out = Softmax4D()(conv)

ensemble = Model(inputs=ensemble_input, outputs=out)

In [9]:
import custom_metrics as M
from keras.optimizers import Adam

class_weights = [1516.576, 0.1]
unsuper_weight = 0.1

loss = M.loss(weights=class_weights, unsuper_weight=unsuper_weight, unsuper_channel=-1)
wcce = M.wcceOA(weights=class_weights)
uentr = M.entrONA(unsuper_channel=-1)
wacc = M.waccOA(weights=class_weights)

ensemble.compile(optimizer=Adam(1e-4), loss=loss, metrics=[wcce, uentr, wacc])

In [10]:
import AFLungNet as LN
import uuid
import os
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger
from keras.utils import plot_model

# save model image
modeldir = 'model-'+str(uuid.uuid4())
print(modeldir)
if not os.path.exists(modeldir):
    os.makedirs(modeldir)
open(modeldir+'/architecture.json', 'w').write(ensemble.to_json())
plot_model(ensemble, to_file=modeldir+'/model.png', show_shapes=True)

train_gen = LN.modified_sample_generator('lidc-train.npz')
val_gen = LN.modified_sample_generator('lidc-val.npz', augment=False)

checkpoints = ModelCheckpoint(modeldir+'/weights.{epoch:02d}-{val_waccOA:.2f}.hdf5', 
                              monitor='val_waccOA', verbose=1, mode='max',
                              save_best_only=True)
earlystopping = EarlyStopping(monitor='val_waccOA', min_delta=0.001, patience=40, 
                              verbose=1, mode='max')
logger = CSVLogger(modeldir+'/training.log')

# fit model
ensemble.fit_generator(train_gen, 400, 1000, validation_data=val_gen, validation_steps=200,
                    callbacks=[checkpoints, earlystopping, logger])

model-27b8ee78-d5aa-4dbd-a4bf-62726e011c16


C:\Users\PC-Komputer\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/1000
400/400 [==============================] - 55s 101ms/step - loss: 3.1213 - wcceOA: 3.4543 - entrONA: 0.1235 - waccOA: 0.6367 - val_loss: 0.7441 - val_wcceOA: 0.7630 - val_entrONA: 0.5747 - val_waccOA: 0.3881

Epoch 00001: val_waccOA improved from -inf to 0.38809, saving model to model-27b8ee78-d5aa-4dbd-a4bf-62726e011c16\weights.01-0.39.hdf5
Epoch 2/1000
400/400 [==============================] - 13s 32ms/step - loss: 0.7092 - wcceOA: 0.7194 - entrONA: 0.6179 - waccOA: 0.5948 - val_loss: 0.6978 - val_wcceOA: 0.7058 - val_entrONA: 0.6254 - val_waccOA: 0.4146

Epoch 00002: val_waccOA improved from 0.38809 to 0.41457, saving model to model-27b8ee78-d5aa-4dbd-a4bf-62726e011c16\weights.02-0.41.hdf5
Epoch 3/1000
400/400 [==============================] - 14s 34ms/step - loss: 0.7100 - wcceOA: 0.7204 - entrONA: 0.6163 - waccOA: 0.6154 - val_loss: 0.6594 - val_wcceOA: 0.6573 - val_entrONA: 0.6786 - val_waccOA: 0.6414

Epoch 00003: val_waccOA improved from 0.41457 to 0.64145, savin

# Podsumowanie
Najlepszy wynik jaki osiągneliśmy po połączeniu na zbiorze walidacyjnym to 0.86 ważonej celności. Zapewne wynika to, z nie najlepszego wyniku sieci *U-Net*.